# IS620 Final Project

Data:  ~4200 pub med journals, including author affilations.  The affiliations are not provided in an specified manner, and are instead long 'strings'.  

Goal:  It is not clear, looking at the strings, how to distinuish instituion names from department names and so on.  The goal here is to develop a maching learning algo which can pull instinution names from the affilations, such that a graph of schools which publish with eachother might be generated.

Results:  A decision tree was found to be most appliciable.  The accuracy was ~92% with the test set.  However, this only tells part of the story; I labeled a training set of ~4000 samples, and some of these I mislabeled.  Thus the machine learning had to properly assign labels that I had put in incorrectly!




In [1]:
import random
import nltk
import string

In [2]:
## Here we open a file of downloaded journal articles including author names and affiliations.
pubs = []
with open('project2pubs.txt','r') as f:
    for line in f:
        pubs.append(line.split(']]'))
        

In [72]:
## As you can see, the data is a mess
pubs[0]

["[['Ma Y', 'Zhang J', 'Yin W', 'Zhang Z', 'Song Y', 'Chang X'], ['Ma Y', 'Zhang J', 'Yin W', 'Zhang Z', 'Song Y', 'Chang X'], ['Ma Y', 'Zhang J', 'Yin W', 'Zhang Z', 'Song Y', 'Chang X'], ['Ma Y', 'Zhang J', 'Yin W', 'Zhang Z', 'Song Y', 'Chang X'], ['Ma Y', 'Zhang J', 'Yin W', 'Zhang Z', 'Song Y', 'Chang X'], ['Ma Y', 'Zhang J', 'Yin W', 'Zhang Z', 'Song Y', 'Chang X'",
 ',Key Laboratory of Stem Cell Biology, Institute of Health Sciences, Shanghai Institutes for Biological Sciences &Shanghai Jiao Tong University School of Medicine (SJTUSM), Chinese Academy of Sciences, Shanghai, China. Key Laboratory of Stem Cell Biology, Institute of Health Sciences, Shanghai Institutes for Biological Sciences &Shanghai Jiao Tong University School of Medicine (SJTUSM), Chinese Academy of Sciences, Shanghai, China. Key Laboratory of Stem Cell Biology, Institute of Health Sciences, Shanghai Institutes for Biological Sciences &Shanghai Jiao Tong University School of Medicine (SJTUSM), Chinese Academy o

In [503]:
## The form of the strings in the files, are a mess, a much effort needed to clean it all up
stringIn = "string.with.punctuation!"

author = []
for x in pubs:
    if isinstance(x, list):
        t = (x[0].translate(None, '!@#$][,').split("'"))
        temp = []
        for x in t: 
            if x:
                if x != " ":
                    if x not in temp:
                       temp.append(x)
        author.append(temp)
            
        
    
author[0]

['Ma Y', 'Zhang J', 'Yin W', 'Zhang Z', 'Song Y', 'Chang X']

In [504]:
len(author)

4215

In [505]:
len(pubs)

4215

In [3]:
## The affiliation data is a mess.  Sometimes its deliminated with . or ;, etcetera.  
## below we split up each affiliation using a number of delimiters

splitcomma = []
splitperiod = []
splitsemicolon = []
splitspace = []
for x in pubs:
    if len(x) > 1:
        s = x[1]
        s = s[:-1]
        t = s.split(",")
        if len(t)>2:
            t = t[:-2]  # remove journal names
        else:
            t = " "
        for x in range(0, len(t)):
            if isinstance(t[x], str):
                t[x] = str(t[x]).lstrip()
                t[x] = str(t[x]).rstrip()
        splitcomma.append(t)
        t = s.split(".")
        for x in range(0, len(t)):
            if isinstance(t[x], str):
                t[x] = str(t[x]).lstrip()
                t[x] = str(t[x]).rstrip()
        splitperiod.append(t)
        t = s.split(";")
        for x in range(0, len(t)):
            if isinstance(t[x], str):
                t[x] = str(t[x]).lstrip()
                t[x] = str(t[x]).rstrip()
        splitsemicolon.append(t)
        t = s.split(" ")
        for x in range(0, len(t)):
            if isinstance(t[x], str):
                t[x] = str(t[x]).lstrip()
                t[x] = str(t[x]).rstrip()
        splitspace.append(t)
    else:
        splitcomma.append("")
        splitperiod.append("")
        splitsemicolon.append("")
        splitspace.append("")
        

    
        


In [4]:
def stringHasPunct(s):
    if not s:
        return True
    for x in string.punctuation:
        if x in s:
            return True
    return False

In [5]:
## below we generate a list of all possible 'phrases' in the affiliation

listterms = []
for x in range(0, len(splitcomma)):
    temp = []
    t = []
    for y in splitcomma[x]:
        if not stringHasPunct(y):
            t.append(y)
    temp.append(t)
    t = []
    for y in splitsemicolon[x]:
        if not stringHasPunct(y):
            t.append(y)
    temp.append(t)
    t = []
    for y in splitperiod[x]:
        if not stringHasPunct(y):
            t.append(y)
    temp.append(t)
    listterms.append(temp)

In [6]:
## example affiliations are shown below.  The question is, can
## a machine learning classification determine which are institution names??
listterms[2]

[['Qinghai Provincial Key Laboratory for Plague Control and Research',
  'Qinghai Institute for Endemic Disease Prevention and Control',
  'Xining',
  'Qinghai Province',
  '811602',
  'Fengtai',
  'Beijing',
  '100071',
  'Qinghai Institute for Endemic Disease Prevention and Control',
  'Xining',
  'Qinghai Province',
  '811602',
  'Fengtai',
  'Beijing',
  '100071'],
 [],
 ['com']]

In [512]:
## I save all the phrases to a csv.  I will label some of the 
## phrases in the csv file to then train the system.
f = open('labelAffiliations.txt','w')
for x in listterms:
    for y in x:
        for z in y:
            f.write(str(z) + '\n') 
f.close() 


In [7]:
## in excel, I marked >4000 'phrases' as belonging to an 
## instituion or not.  Most of these were just zip codes and such,
## so easy to mark.

phrase = []
label = []
with open('labelAffiliations.csv') as f:
    for line in f:
        line = line.rstrip()
        line = line.split(",")
        if len(line) >1 and line[1]:
            phrase.append(line[0])
            label.append(line[1])
            
            

In [8]:
len(label)

4866

In [9]:
## get all words used in any labeled affiliation
words = []
for x in phrase:
    x = x.split(" ")
    for y in x:
        words.append(y)
        

In [10]:
len(words)

15304

In [11]:
#look at the top 200 words
all_words = nltk.FreqDist(words)
top100 = all_words.most_common(50)

In [12]:
word_feature = []
for x in top100:
    word_feature.append(x[0])

In [13]:
word_feature

['of',
 'and',
 'Department',
 'for',
 'Center',
 'Biology',
 'University',
 'Institute',
 'Research',
 'Sciences',
 'Medicine',
 'Division',
 'Science',
 'Molecular',
 'Cell',
 'Medical',
 'Centre',
 'College',
 'Cancer',
 'Street',
 'de',
 'Genetics',
 'Laboratory',
 'Road',
 'Engineering',
 'Avenue',
 'National',
 'in',
 'Beijing',
 'Microbiology',
 'Technology',
 'New',
 'Health',
 'Biological',
 'Biotechnology',
 'Drive',
 'School',
 'China',
 'CA',
 'Cambridge',
 'Biochemistry',
 'Biol',
 'a',
 'the',
 'York',
 'Life',
 'Diseases',
 'California',
 '1',
 'Biomedical']

In [14]:
listterms[0]

[['Key Laboratory of Stem Cell Biology',
  'Institute of Health Sciences',
  'Chinese Academy of Sciences',
  'Shanghai',
  'Institute of Health Sciences',
  'Chinese Academy of Sciences',
  'Shanghai',
  'Institute of Health Sciences',
  'Chinese Academy of Sciences',
  'Shanghai',
  'Institute of Health Sciences',
  'Chinese Academy of Sciences',
  'Shanghai',
  'University of California at San Diego',
  'La Jolla',
  'La Jolla',
  'California',
  'Institute of Health Sciences',
  'Chinese Academy of Sciences',
  'Shanghai',
  'Shanghai Jiao Tong University School of Medicine',
  'Shanghai'],
 [],
 []]

In [15]:
## the feature I want to look at.  

def getContentIndex(listterms_x, phrase):
    '''what affiliation does a phrase occur in.
    Bear in mind that I labeled phrases free of any context
    and now need to figure out which affilation each phrase come from'''
    for x in listterms_x: ## x is a list ['ewfew', 'wefew' ]
        if phrase in x:
            return x.index(phrase), x
    else:
        return False, False

def hasNumber(word):
    for x in word:
        if is_number(x):
            return True
    return False

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def firstWordisCapital(x):
    fl = x[0]
    if is_number(fl):
        return False
    if fl == fl.upper():
        return True
    return False

def ratioOfCaptialLetters(phrase):
    p = phrase.split()
    c = 0
    for x in p:
        for y in x:
            if not is_number(y):
                if y == y.upper():
                    c += 1
    if len(p) != 0:
        r = c/float(len(p))
    else:
        r = 0
    return int(r*10)

def numberOfWords(phrase):
    p = phrase.split()
    return int(len(p)/2)

def oneWord(phrase):
    p = phrase.split()
    if len(p) == 1:
        return True
    else:
        return False
    

def totalLen(phrase):
    return int(len(phrase)/10)
    
                
    

In [16]:
for word in word_feature:
        (word in set())

In [20]:
def document_features_old(context, phrase, word_feature):
    doci, i = getContentIndex(context, phrase)
    
    context = context[i]
    document_words = set(phrase.split())
    features = {}
    for word in word_feature:
        features['contains(%s)' % word] = (word in document_words)
    features['isnumeric'] = is_number(phrase)
    features['hasANumberInIt'] = hasNumber(phrase)
    features['firstLetterCapital'] = firstWordisCapital(phrase)
    features['ratioOfCaptialLetters'] = ratioOfCaptialLetters(phrase)
    features['numberOfWords'] = numberOfWords(phrase)
    features['startsWithNumber'] = is_number(phrase[0])
    features['totalLength'] = totalLen(phrase)
    features['oneWord'] = oneWord(phrase)
    
    if doci != 0:
        phrasebefore = context[doci-1]
        document_words = set(phrasebefore)
        #for word in word_feature:
            #features['phraseBeforeContains(%s)' % word] = (word in set(document_words))
        features['isnumericBefore'] = is_number(phrasebefore)
        features['hasANumberInItBefore'] = hasNumber(phrasebefore)
        features['firstLetterCapitalBefore'] = firstWordisCapital(phrasebefore)
        features['ratioOfCaptialLettersBefore'] = ratioOfCaptialLetters(phrasebefore)
        features['numberOfWordsBefore'] = numberOfWords(phrasebefore)
        features['startsWithNumberBefore'] = is_number(phrasebefore[0])
        features['totalLengthBefore'] = totalLen(phrasebefore)
        features['oneWordBefore'] = oneWord(phrasebefore)
        
    
    if doci + 1 < len(context):
        phraseafter = context[doci+1]
        document_words = set(phraseafter)
        #for word in word_feature:
            #features['phraseAfterContains(%s)' % word] = (word in set(document_words))
        features['isnumericAfter'] = is_number(phraseafter)
        features['hasANumberInItAfter'] = hasNumber(phraseafter)
        features['firstLetterCapitalAfter'] = firstWordisCapital(phraseafter)
        features['ratioOfCaptialLettersAfter'] = ratioOfCaptialLetters(phraseafter)
        features['numberOfWordsAfter'] = numberOfWords(phraseafter)
        features['startsWithNumberAfter'] = is_number(phraseafter[0])
        features['totalLengthAfter'] = totalLen(phraseafter)
        features['oneWordAfter'] = oneWord(phraseafter)
    if doci + 2 < len(context):
        phraseafter = context[doci+2]
        document_words = set(phraseafter)
        #for word in word_feature:
            #features['phraseAfter2Contains(%s)' % word] = (word in set(document_words))
        features['isnumericAfter2'] = is_number(phraseafter)
        features['hasANumberInItAfter2'] = hasNumber(phraseafter)
        features['firstLetterCapitalAfter2'] = firstWordisCapital(phraseafter)
        features['ratioOfCaptialLettersAfter2'] = ratioOfCaptialLetters(phraseafter)
        features['numberOfWordsAfter2'] = numberOfWords(phrase)
        features['totalLengthAfter2'] = totalLen(phraseafter)
        features['oneWordAfter2'] = oneWord(phraseafter)
    if doci + 3 < len(context):
        phraseafter = context[doci+3]
        document_words = set(phraseafter)
        #for word in word_feature:
            #features['phraseAfter3Contains(%s)' % word] = (word in set(document_words))
        features['isnumericAfter3'] = is_number(phraseafter)
        features['hasANumberInItAfter3'] = hasNumber(phraseafter)
        features['firstLetterCapitalAfter3'] = firstWordisCapital(phraseafter)
        features['ratioOfCaptialLettersAfter3'] = ratioOfCaptialLetters(phraseafter)
        features['numberOfWordsAfter3'] = numberOfWords(phraseafter)
        features['totalLengthAfter3'] = totalLen(phraseafter)
        
        
    return features

In [60]:
def document_features(context, phrase, word_feature):
    ## context is the 'listterms' which is a list of 3 lists

    doci, context = getContentIndex(context, phrase)
    document_words = set(phrase.split())
    features = {}
    for word in word_feature:
        features['contains(%s)' % word] = (word in document_words)
    features['isnumeric'] = is_number(phrase)
    features['hasANumberInIt'] = hasNumber(phrase)
    features['firstLetterCapital'] = firstWordisCapital(phrase)
    features['ratioOfCaptialLetters'] = ratioOfCaptialLetters(phrase)
    features['numberOfWords'] = numberOfWords(phrase)
    features['startsWithNumber'] = is_number(phrase[0])
    features['totalLength'] = totalLen(phrase)
    features['oneWord'] = oneWord(phrase)
    
    if doci != 0:
        phrasebefore = context[doci-1]
        document_words = set(phrasebefore.split())
        #for word in word_feature:
            #features['phraseBeforeContains(%s)' % word] = (word in set(document_words))
        features['isnumericBefore'] = is_number(phrasebefore)
        features['hasANumberInItBefore'] = hasNumber(phrasebefore)
        features['firstLetterCapitalBefore'] = firstWordisCapital(phrasebefore)
        features['ratioOfCaptialLettersBefore'] = ratioOfCaptialLetters(phrasebefore)
        features['numberOfWordsBefore'] = numberOfWords(phrasebefore)
        features['startsWithNumberBefore'] = is_number(phrasebefore[0])
        features['totalLengthBefore'] = totalLen(phrasebefore)
        features['oneWordBefore'] = oneWord(phrasebefore)
    if doci + 1 < len(context):
        phraseafter = context[doci+1]
        document_words = set(phraseafter.split())
        #for word in word_feature:
            #features['phraseAfterContains(%s)' % word] = (word in set(document_words))
        features['isnumericAfter'] = is_number(phraseafter)
        features['hasANumberInItAfter'] = hasNumber(phraseafter)
        features['firstLetterCapitalAfter'] = firstWordisCapital(phraseafter)
        features['ratioOfCaptialLettersAfter'] = ratioOfCaptialLetters(phraseafter)
        features['numberOfWordsAfter'] = numberOfWords(phraseafter)
        features['startsWithNumberAfter'] = is_number(phraseafter[0])
        features['totalLengthAfter'] = totalLen(phraseafter)
        features['oneWordAfter'] = oneWord(phraseafter)
    return features

In [71]:
def document_features(context, phrase, word_feature):
    ## context is the 'listterms' which is a list of 3 lists

    document_words = set(phrase.split())
    features = {}
    for word in word_feature:
        features['contains(%s)' % word] = (word in document_words)
    features['isnumeric'] = is_number(phrase)
    features['hasANumberInIt'] = hasNumber(phrase)
    features['firstLetterCapital'] = firstWordisCapital(phrase)
    features['ratioOfCaptialLetters'] = ratioOfCaptialLetters(phrase)
    features['numberOfWords'] = numberOfWords(phrase)
    features['startsWithNumber'] = is_number(phrase[0])
    features['totalLength'] = totalLen(phrase)
    features['oneWord'] = oneWord(phrase)
    return features

In [75]:
d_phrases = {}
for x in range(0, len(phrase)):
    d_phrases[phrase[x]] = label[x]

In [76]:
random.shuffle(listterms)
len(d_phrases)

4177

In [77]:
listterms[0] ## each list has 3 lists (split by comma, semicolon, etc...)

[['Graduate School of Arts and Sciences',
  'The University of Tokyo',
  'The University of Tokyo',
  'The University of Tokyo',
  'The University of Tokyo',
  'The University of Tokyo'],
 [],
 ['ecc', 'ac', 'jp']]

In [78]:
featuresets = []
phrasesLookedUp = []
for x in listterms:
    for y in x:
        for z in y:
            if z in d_phrases:
                featuresets.append([document_features(x, z, word_feature), d_phrases[z]]) 
                phrasesLookedUp.append(z)


In [79]:
train_set, test_set = featuresets[2500:], featuresets[:2500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [80]:
## achieve 90% accuracy
print nltk.classify.accuracy(classifier, test_set)

0.9528


In [81]:
## 92% of the test set are not instituions.  At first this seems like bad news.
## But look at errors below
c = 0
for x in test_set:
    if x[1] == 's':
        c += 1
1-float(c)/len(test_set)

0.9236

In [82]:
classifier.show_most_informative_features(30)

Most Informative Features
   ratioOfCaptialLetters = 6                   g : s      =     76.9 : 1.0
       contains(Biology) = True                g : s      =     62.5 : 1.0
      contains(Sciences) = True                g : s      =     58.9 : 1.0
    contains(Technology) = True                s : g      =     40.9 : 1.0
    contains(University) = True                s : g      =     24.4 : 1.0
           numberOfWords = 4                   g : s      =     21.6 : 1.0
             totalLength = 6                   g : s      =     18.5 : 1.0
           contains(for) = True                g : s      =     15.1 : 1.0
        contains(School) = True                s : g      =     12.7 : 1.0
        contains(Health) = True                s : g      =     12.5 : 1.0
          contains(Life) = True                g : s      =     11.2 : 1.0
      contains(Research) = True                g : s      =     10.6 : 1.0
           numberOfWords = 5                   g : s      =      9.6 : 1.0

In [83]:
errors = []
i = 0
for (name, tag) in test_set:
    guess = classifier.classify(name)
    phrase1 = phrasesLookedUp[i]
    if guess != tag:
        errors.append( (tag, guess, phrase1) )
    i += 1

In [59]:
## My main erors appear to be mis labeled as instituions --by me!-- and vice versa
## and the classifier has correctly labeled them..
## However, the classifier also likes to label cities as schools, despite they being a 
## single word...
list(set(errors))

[('g', 's', 'David H'),
 ('g', 's', 'Broad Institute of MIT and Harvard'),
 ('g', 's', 'Academy of Military Medical Sciences'),
 ('g', 's', 'Beijing University of Technology'),
 ('g', 's', 'Campus Giessen'),
 ('g', 's', 'Atlanta'),
 ('s', 'g', 'David Geffen School of Medicine'),
 ('g', 's', 'College of Veterinary Medicine'),
 ('s', 'g', 'Massachusetts General Hospital'),
 ('g', 's', 'Berlin'),
 ('g', 's', 'Chinese Academy of Medical Sciences'),
 ('g', 's', 'Birmingham VA Medical Center'),
 ('g', 's', 'Anhui University'),
 ('g', 's', 'Boston'),
 ('g', 's', 'Columbus'),
 ('g', 's', 'Athens'),
 ('g', 's', 'Danish Technical University'),
 ('g', 's', 'Brisbane'),
 ('g', 's', 'China CUHK Shenzhen Research Institute'),
 ('g', 's', 'Atlanta VA Medical Center'),
 ('g', 's', 'Bloomington'),
 ('g', 's', 'Churchill Hospital'),
 ('g', 's', 'Campus de San Vicente'),
 ('g', 's', 'Department of Veterinary Medicine'),
 ('g', 's', 'Biosci Biotechnol Biochem'),
 ('s', 'g', 'Berkeley'),
 ('s', 'g', 'Mayo 

In [451]:
## below is used to see what is marked as an institution
sguess = []
i = 0
for (name, tag) in test_set:
    guess = classifier.classify(name)
    phrase2 = phrasesLookedUp[i]
    if tag == 's':
        sguess.append( (tag, guess, phrase2) )
    i += 1

In [86]:
## lets see how a decision tree does
classifier_dt = nltk.classify.DecisionTreeClassifier.train(train_set)

In [87]:
## this error rate looks a bit better than for the bayes.
print nltk.classify.accuracy(classifier_dt, test_set)

0.9696


In [88]:
## I see a lot of reasonable corrections to my assignments, so the 
## classifier is doing a better job than I.  Plus I don't see
## cities being named as institutions.  

errors = []
i = 0
for (name, tag) in test_set:
    guess = classifier.classify(name)
    phrase3 = phrasesLookedUp[i]
    if guess != tag:
        errors.append( (tag, guess, phrase3) )
    i += 1

list(set(errors))

[('g', 's', 'Cardiff University School of Medicine'),
 ('g', 's', 'China National Rice Research Institute'),
 ('g', 's', 'College of Veterinary Medicine'),
 ('g', 's', 'Animal Nutrition and Health'),
 ('g', 's', 'Alfaisal University'),
 ('g', 's', 'Charite Medical University Berlin'),
 ('g', 's', 'Broad Institute of MIT and Harvard'),
 ('g', 's', 'China Shanghai Institute of Pharmaceutical Industry'),
 ('g', 's', 'Nottingham University Hospitals NHS Trust'),
 ('s', 'g', 'Denmark Max Planck Institute for Informatics'),
 ('g', 's', 'Danish Technical University'),
 ('g', 's', 'College of Basic Medicine'),
 ('g', 's', 'Anhui Agricultural University'),
 ('s', 'g', 'Berkeley'),
 ('s', 'g', 'Mayo Clinic'),
 ('g', 's', 'Beth Israel Deaconess Medical Center'),
 ('g', 's', 'National Institute of Neuroscience'),
 ('g', 's', 'Dalhousie University'),
 ('g', 's', 'Central South University'),
 ('g', 's', 'Brandeis University National Center for Behavioral Genomics'),
 ('g', 's', 'Baylor College of Me

In [464]:
## At this point, I think the decision tree is better

In [40]:
def splitIntoPhrases(affiliation):
    '''this long function is need to take each affilation string, and break it down into all its 
    possible deliminated pieces, and then take those and convert it to a list of 'phrases', one or more 
    of these phrases being a poissible institution name'''
    
    splitcomma = []
    splitperiod = []
    splitsemicolon = []
    splitspace = []
    if len(affiliation) > 1:
        #print (x)
        s = affiliation[1]
        s = s[:-1]
        
        t = s.split(",")
        if len(t)>2:
            t = t[:-2]  # remove journal names
        else:
            t = " "
        for x in range(0, len(t)):
            if isinstance(t[x], str):
                t[x] = str(t[x]).lstrip()
                t[x] = str(t[x]).rstrip()
        splitcomma.append(t)
        t = s.split(".")
        for x in range(0, len(t)):
            if isinstance(t[x], str):
                t[x] = str(t[x]).lstrip()
                t[x] = str(t[x]).rstrip()
        splitperiod.append(t)
        t = s.split(";")
        for x in range(0, len(t)):
            if isinstance(t[x], str):
                t[x] = str(t[x]).lstrip()
                t[x] = str(t[x]).rstrip()
        splitsemicolon.append(t)
        t = s.split(" ")
        for x in range(0, len(t)):
            if isinstance(t[x], str):
                t[x] = str(t[x]).lstrip()
                t[x] = str(t[x]).rstrip()
        splitspace.append(t)
    else:
        splitcomma.append("")
        splitperiod.append("")
        splitsemicolon.append("")
        splitspace.append("")  ## not used
    listterms = []
    
    for x in range(0, len(splitcomma)):
        temp = []
        t = []
        for y in splitcomma[x]:
            if not stringHasPunct(y):
                t.append(y)
        temp.append(t)
        t = []
        for y in splitsemicolon[x]:
            if not stringHasPunct(y):
                t.append(y)
        temp.append(t)
        t = []
        for y in splitperiod[x]:
            if not stringHasPunct(y):
                t.append(y)
        temp.append(t)
        listterms.append(temp)
    return listterms

In [89]:
# I might as well train now on the full set of labeled data.
classifier = nltk.NaiveBayesClassifier.train(featuresets)
classifier_dt = nltk.classify.DecisionTreeClassifier.train(featuresets)

In [92]:
c = 0
predictions = []
for x in pubs:
    listphrases = splitIntoPhrases(x)
    t = []
    for i in listphrases:
        for j in i:
            for k in j:
                p = classifier_dt.classify(document_features(x, k, word_feature))
                if p == 's':  
                    t.append(k)
                else:
                    t.append('NA')
    predictions.append(t)
    c += 1

In [93]:
affiliations = []
for x in predictions:
    affiliations.append(list(set(x)))

In [94]:
for x in affiliations:
    if 'NA' in x:
        x.remove('NA')

In [95]:
affiliations[:10]

[['Shanghai Jiao Tong University School of Medicine',
  'University of California at San Diego'],
 [],
 [],
 ['University of Pennsylvania Perelman School of Medicine',
  'Michigan State University',
  'Kawasaki Medical School',
  'Graduate School of Medicine',
  'Kyoto University'],
 [],
 ['Creighton University'],
 ['Wageningen University', 'Faculty of Medicine University of Helsinki'],
 [],
 ['Tamagawa University', 'The University of Tokyo'],
 ['University of Turku and Abo Academy']]

In [101]:
g1 = []
for x in affiliations:
    if len(x)>1:
        for i in range(0, len(x)):
            for j in range(i+1, len(x)):
                g1.append([x[i], x[j]])

In [104]:
## Now save the undirected graph nodes
f = open('IS620Project3.csv','w')
for x in g1:
    f.write(str(x[0]) + "," +  str(x[1]) + '\n') 
f.close() 
